# MAST30034 Project 1
## Analysis

In [174]:
from functools import reduce 
from pyspark.sql import DataFrame
from pyspark.sql import SparkSession

# Create a spark session (which will run spark jobs)
spark = (
    SparkSession.builder.appName("Project 1 - Preprocessing")
    .config("spark.sql.repl.eagerEval.enabled", True) 
    .config("spark.sql.parquet.cacheMetadata", "true")
    .getOrCreate()
)

In [175]:
import pandas as pd
import geopandas as gpd
import folium

### Geopandas visualisation
#### Average tip earnings per taxi zone

In [176]:
sf = gpd.read_file("/Users/jackmelleuish/Library/CloudStorage/OneDrive-TheUniversityofMelbourne/Desktop/Jack/Uni/2022/Semester 2/Data Science/Project 1/taxi_zones/taxi_zones.shp")
zones = pd.read_csv("/Users/jackmelleuish/Library/CloudStorage/OneDrive-TheUniversityofMelbourne/Desktop/Jack/Uni/2022/Semester 2/Data Science/Project 1/taxi_zones/taxi+_zone_lookup.csv")
sf['geometry'] = sf['geometry'].to_crs("+proj=longlat +ellps=WGS84 +datum=WGS84 +no_defs")
gdf = gpd.GeoDataFrame(
    pd.merge(zones, sf, on='LocationID', how='inner')
)
geoJSON = gdf[['LocationID', 'geometry']].drop_duplicates('LocationID').to_json()


In [178]:
df = spark.read.parquet('DataFrames/geo_tip_df')
df = df.toPandas()
df = df \
    .merge(gdf[['LocationID', 'geometry']], left_on='pickup_location', right_on='LocationID') \
    .drop('LocationID', axis=1)

In [180]:
df

,pickup_location,total_trips,total_tips,avg_tip_amount,geometry
0,29,12204,5881.95,0.481969,"POLYGON ((-73.96005 40.58327, -73.95901 40.582..."
1,26,23857,6496.42,0.272307,"POLYGON ((-73.98332 40.64148, -73.98299 40.641..."
2,65,211006,492434.98,2.333749,"POLYGON ((-73.98712 40.70063, -73.98706 40.700..."
3,191,11438,2329.10,0.203628,"POLYGON ((-73.73017 40.72396, -73.72991 40.723..."
4,222,14557,3627.65,0.249203,"MULTIPOLYGON (((-73.87909 40.65020, -73.87844 ..."
...,...,...,...,...,...
253,2,5,3.56,0.712000,"MULTIPOLYGON (((-73.82338 40.63899, -73.82277 ..."
254,158,6,8.56,1.426667,"POLYGON ((-74.00779 40.74197, -74.00697 40.741..."
255,114,15,11.03,0.735333,"POLYGON ((-73.99258 40.72420, -73.99260 40.724..."
256,211,2,0.00,0.000000,"POLYGON ((-74.00152 40.71982, -74.00188 40.719..."


In [179]:
map = folium.Map(location=[40.73, -73.74], tiles="Stamen Terrain", zoom_start=10)

# refer to the folium documentations on more information on how to plot aggregated data.
c = folium.Choropleth(
    geo_data=geoJSON, # geoJSON 
    name='Average Tip Earnings (USD)', # name of plot
    data=non_zero_tips, # data source
    columns=['pickup_location','avg_tip_amount'], # the columns required
    key_on='properties.LocationID', # this is from the geoJSON's properties
    fill_color='YlOrRd', # color scheme
    nan_fill_color='black',
    legend_name='Average Tip Earnings (USD)'
)
c.add_to(map)
map.save("outputs/Average_tip_Amount.html")

In [ ]:
'''
payment_method = df[['payment_type', 'tip_amount']] \
                .groupby('payment_type') \
                .agg(
                    {
                        'tip_amount': 'sum', # sum over total amount earned
                        'payment_type': 'count' # count number of instances from sample
                    }
                ) \
                .rename({'payment_type': 'total_trips', 'tip_amount': 'total_tips'}, axis=1)
payment_method['avg_tip_amount'] = payment_method['total_tips'] / payment_method['total_trips']
payment_method = payment_method.reset_index(level=0)
payment_method = payment_method.drop([3, 4])
payment_method
plot = payment_method.plot.bar(x = "payment_type", y = 'avg_tip_amount', logy = True).set_xticklabels(["card", "cash", "No charge"])
'''

### Linear Modelling